Camel 3 Function 
=====================


In [4]:
include("notebook_setup.jl")

🚀 Setting up GlobtimPlots notebook environment
📦 Activating Globtim environment at: /Users/ghscholt/globtim


  Activating 

📦 Adding GlobtimPlots in dev mode from: /Users/ghscholt/globtimplots/


project at `~/globtim`
   Resolving package versions...



📋 Loading required packages...
  ✅ Globtim
  ✅ GlobtimPlots
  ✅ CairoMakie
  ✅ DataFrames

✅ Notebook setup complete!
📋 Available from Globtim:
  - camel function
  - test_input constructor
  - Constructor for polynomial approximation
  - RationalPrecision
  - solve_polynomial_system
  - process_crit_pts
  - analyze_critical_points
📋 Available from GlobtimPlots:
  - cairo_plot_polyapprox_levelset
  - AbstractPolynomialData, AbstractProblemInput
  - adapt_polynomial_data, adapt_problem_input


  No Changes to `~/globtim/Project.toml`
  No Changes to `~/globtim/Manifest.toml`


We define the center and range of the square domain of approximation. 

In [5]:
# Constants and Parameters
const n, a, b = 2, 5, 1
const scale_factor = a / b
f = camel # Objective function
d = 6 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

Number of samples: 40000


test_input(2, [0.0, 0.0], 200, (0.1, 0.5), 0.002, (0.0, 0.0), 5.0, 1.0, 6, Globtim.camel)

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision. 

In [6]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

current L2-norm: 5.4524172392659695e-12
current L2-norm: 4.6461539462782044e-12


Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [ ]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

In [ ]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.00125);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.00125);

In [ ]:
fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb, chebyshev_levels=true)

In [ ]:
fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true)
# Commented out save command
# save("camel_lege_levelset.pdf", fig_2)